# Genios.de Artikel-Scraper (Deutsch - Rev. 2)

Dieses Skript extrahiert Artikel-Metadaten von Genios.de für einen bestimmten Suchbegriff und Zeitraum.
Aufgrund einer Begrenzung von 10.000 Ergebnissen pro Abfrage iteriert es Tag für Tag, um alle Ergebnisse zu sammeln.

**Haftungsausschluss:**
Web-Scraping sollte verantwortungsbewusst und ethisch korrekt durchgeführt werden.
- Überprüfen Sie immer die `robots.txt` der Webseite (z.B. `https://www.genios.de/robots.txt`) und deren Nutzungsbedingungen.
- Überlasten Sie den Server nicht; implementieren Sie Verzögerungen zwischen den Anfragen.
- Dieses Skript dient Bildungszwecken und setzt die Einhaltung geltender Gesetze voraus, wie z.B. § 60d UrhG (Text und Data Mining für wissenschaftliche Forschung), falls anwendbar.

Das Skript extrahiert:
- Artikel-ID (Attribut)
- Dokumenten-ID (Attribut)
- Titel
- Snippet (Teaser-Text)
- Trefferumgebung / Kontext-Snippets (Kontext des Suchbegriffs, robuster extrahiert)
- Quellenname (z.B. Frankfurter Allgemeine Sonntagszeitung (FAS))
- Quellen-Kürzel (z.B. FAS)
- Artikeldatum
- Artikel-URL (Link zur Genios-Dokumentenseite)
- Wortanzahl
- Preis (falls verfügbar)
- Vorschaubild-URL
- Abfragedatum (das Datum, für das die Suche durchgeführt wurde)

und speichert alles in einer CSV-Datei.

In [ ]:
import requests
from bs4 import BeautifulSoup, NavigableString
import time
import random
from datetime import datetime, timedelta, date
import csv
from urllib.parse import urljoin
import os
import re

## Konfiguration
Stellen Sie hier die Parameter für das Scraping ein.

In [ ]:
SUCHBEGRIFF = "AFD"
# Datumsbereich (einschließlich)
START_DATUM_STR = "01.09.2024"
ENDE_DATUM_STR = "01.09.2024" # Für Tests ggf. zuerst einen kleineren Bereich wählen, z.B. nur einen Tag

# Basis-URL für Suchergebnisse
BASIS_URL = "https://www.genios.de/searchResult/Alle/Presse"

# Basis-Parameter für die Suche
BASIS_PARAMETER = {
    'requestText': SUCHBEGRIFF,
    'category': 'Presse Deutschland',
    'size': 100,  # Anzahl Ergebnisse pro Seite
    'sort': 'BY_DATE',
    'order': 'desc',
    'resultListType': 'DEFAULT',
    'view': 'list'
}

# Name der Ausgabe-CSV-Datei
zeitstempel = datetime.now().strftime("%Y%m%d_%H%M%S")
AUSGABE_CSV_DATEI = f"genios_artikel_{SUCHBEGRIFF.lower()}_{zeitstempel}.csv"

# HTTP-Header, um einen Browser zu simulieren
HTTP_HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Verzögerung zwischen Anfragen (in Sekunden)
MIN_VERZOEGERUNG = 1.5
MAX_VERZOEGERUNG = 3.5

# CSV Spaltennamen (wichtig für konsistente Reihenfolge)
CSV_SPALTENNAMEN = [
    'article_id_attr', 'document_id_attr', 'checkbox_value_id',
    'title', 'snippet', 'context_snippet',
    'source_name_full', 'source_name_short', 'source_icon_url',
    'date_published', 'article_url', 'word_count',
    'price', 'preview_image_url', 'query_date' # Das Datum, für das die Abfrage gemacht wurde
]

## Hilfsfunktionen

In [ ]:
def generiere_datumsbereich(start_str, ende_str):
    """Generiert eine Liste von Daten (dd.mm.yyyy) zwischen Start- und Enddatum (einschließlich)."""
    start_dt = datetime.strptime(start_str, "%d.%m.%Y")
    ende_dt = datetime.strptime(ende_str, "%d.%m.%Y")
    delta = timedelta(days=1)
    daten_liste = []
    aktuelles_datum_dt = start_dt
    while aktuelles_datum_dt <= ende_dt:
        daten_liste.append(aktuelles_datum_dt.strftime("%d.%m.%Y"))
        aktuelles_datum_dt += delta
    return daten_liste

def hole_text_sicher(element, standard=""):
    """Extrahiert sicher Text von einem BeautifulSoup-Element."""
    return element.get_text(strip=True) if element else standard

def hole_attribut_sicher(element, attribut, standard=""):
    """Extrahiert sicher ein Attribut von einem BeautifulSoup-Element."""
    return element[attribut] if element and element.has_attr(attribut) else standard

def get_zuletzt_gescrapetes_datum(csv_pfad, datums_spalte='query_date', datums_format='%d.%m.%Y'):
    """Liest die CSV-Datei und gibt das letzte verarbeitete 'query_date' zurück oder None."""
    if not os.path.exists(csv_pfad):
        return None
    
    letztes_datum = None
    try:
        with open(csv_pfad, 'r', newline='', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            if datums_spalte not in reader.fieldnames:
                print(f"Warnung: Spalte '{datums_spalte}' nicht in CSV gefunden für Fortsetzungslogik.")
                return None
            
            gefundene_daten = []
            for zeile in reader:
                datum_str = zeile.get(datums_spalte)
                if datum_str:
                    try:
                        gefundene_daten.append(datetime.strptime(datum_str, datums_format).date())
                    except ValueError:
                        print(f"Warnung: Konnte Datum '{datum_str}' nicht parsen in {csv_pfad}")
            if gefundene_daten:
                letztes_datum = max(gefundene_daten)
    except Exception as e:
        print(f"Fehler beim Lesen der CSV für Fortsetzungslogik: {e}")
        return None 
    
    return letztes_datum.strftime(datums_format) if letztes_datum else None

def bereinige_snippet_text(text):
    """Bereinigt extrahierten Snippet-Text."""
    if not text: return ""
    # Entferne führende/abschließende '..' und überflüssige Leerzeichen
    text = re.sub(r"^\.\.(.*?)\.\.$", r"\1", text.strip()) # Für ..Text..
    text = re.sub(r"^\.\.", "", text.strip()) # Für ..Text
    text = re.sub(r"\.\.$", "", text.strip()) # Für Text..
    text = re.sub(r'\s+', ' ', text).strip() # Normalisiere Leerzeichen
    return text

## Funktionen für das Scraping

In [ ]:
def extrahiere_artikel_daten(artikel_html_element, basis_url_fuer_joins, abgefragtes_datum_str):
    """Extrahiert Daten aus einem einzelnen <article>-HTML-Element."""
    daten = {'query_date': abgefragtes_datum_str}

    daten['article_id_attr'] = hole_attribut_sicher(artikel_html_element, 'id')
    daten['document_id_attr'] = hole_attribut_sicher(artikel_html_element, 'data-document-id')
    
    checkbox = artikel_html_element.find('input', class_='article__checkbox')
    daten['checkbox_value_id'] = hole_attribut_sicher(checkbox, 'value')
    daten['source_name_short'] = hole_attribut_sicher(checkbox, 'data-source')
    
    img_tag_main_div = artikel_html_element.find('div', class_='article__img')
    if img_tag_main_div:
        img_tag_main = img_tag_main_div.find('img', class_='preview-image')
        preview_img_src = hole_attribut_sicher(img_tag_main, 'src')
        daten['preview_image_url'] = urljoin(basis_url_fuer_joins, preview_img_src) if preview_img_src else ""
    else:
        daten['preview_image_url'] = ""
    
    text_bereich = artikel_html_element.find('div', class_='article__text')
    if text_bereich:
        titel_div = text_bereich.find('div', class_='article__text__title')
        daten['title'] = hole_text_sicher(titel_div)

        panel_body_details = text_bereich.find('a', class_='article__text__panelBody__details')
        if panel_body_details:
            # Snippet (Teaser)
            snippet_div = panel_body_details.find('div', class_='article__text__text')
            daten['snippet'] = bereinige_snippet_text(hole_text_sicher(snippet_div))

            # Kontext-Snippet (Trefferumgebung) - Robuster Ansatz
            gefundene_kontexte = []
            # 1. Spezifisches Div suchen
            kontext_div_spezifisch = panel_body_details.find('div', class_='article__text__trefferumgebung')
            if kontext_div_spezifisch:
                gefundene_kontexte.append(bereinige_snippet_text(hole_text_sicher(kontext_div_spezifisch)))
            
            # 2. Generell nach <mark>-Tags im gesamten panelBody suchen, falls spezifisches Div nicht ausreicht oder fehlt
            #    Dies kann zu Duplikaten führen, wenn 'trefferumgebung' bereits <mark> enthält, daher nachher Deduplizierung.
            for elem_mit_mark in panel_body_details.find_all(lambda tag: tag.name == 'div' and tag.find('mark', class_='highlight')):
                # Wir wollen nicht das 'article__text__title' oder 'article__text__text' erneut als Kontext, 
                # wenn es nur um die Trefferumgebung geht.
                if 'article__text__title' not in elem_mit_mark.get('class', []) and \
                   'article__text__text' not in elem_mit_mark.get('class', []):
                    text_content = bereinige_snippet_text(elem_mit_mark.get_text(separator=' ', strip=True))
                    if text_content and text_content not in gefundene_kontexte: # Nur hinzufügen, wenn nicht schon vom spezifischen Div erfasst
                         gefundene_kontexte.append(text_content)
            
            # Wenn keine spezifische Trefferumgebung da war, aber der normale Snippet Highlights hat, diesen auch nehmen
            if not kontext_div_spezifisch and snippet_div and snippet_div.find('mark', class_='highlight'):
                highlight_text = bereinige_snippet_text(hole_text_sicher(snippet_div))
                if highlight_text not in gefundene_kontexte:
                     gefundene_kontexte.append(highlight_text)

            daten['context_snippet'] = " ..//.. ".join(filter(None, gefundene_kontexte)) # Mit Trenner verbinden, leere Strings filtern
        else:
            daten['snippet'] = ""
            daten['context_snippet'] = ""

        link_tag_header = text_bereich.find('a', class_='article__text__panelHeader')
        artikel_href = hole_attribut_sicher(link_tag_header, 'href')
        daten['article_url'] = urljoin(basis_url_fuer_joins, artikel_href) if artikel_href else ""

        if link_tag_header:
            quelle_mit_icon_div = link_tag_header.find('div', class_='article__img__source')
            if quelle_mit_icon_div and quelle_mit_icon_div.find('span'):
                daten['source_name_full'] = hole_text_sicher(quelle_mit_icon_div.find('span'))
                quelle_icon_tag = quelle_mit_icon_div.find('img', class_='article__text__source-icon')
                icon_src = hole_attribut_sicher(quelle_icon_tag, 'src')
                daten['source_icon_url'] = urljoin(basis_url_fuer_joins, icon_src) if icon_src else ""
            else:
                quelle_nur_text_div = link_tag_header.find('div', class_='article__text__source')
                if quelle_nur_text_div:
                    voller_text = hole_text_sicher(quelle_nur_text_div)
                    daten['source_name_full'] = voller_text.split('/')[0].strip() if '/' in voller_text else voller_text
                else:
                    daten['source_name_full'] = daten.get('source_name_short', '')
                daten['source_icon_url'] = "" # Kein Icon in diesem Pfad erwartet
        else:
            daten['source_name_full'] = daten.get('source_name_short', '')
            daten['source_icon_url'] = ""

        text_bar_div = text_bereich.find('div', class_='article__text__bar')
        if text_bar_div:
            wortanzahl_div = text_bar_div.find('div', class_='article__text__bar__number_pages')
            daten['word_count'] = hole_text_sicher(wortanzahl_div).replace('\xa0', ' ').strip()
            kauf_button_container = text_bar_div.find('div', class_='tooltip-button')
            if kauf_button_container:
                 kauf_button = kauf_button_container.find('a', class_='buy_button')
                 daten['price'] = hole_attribut_sicher(kauf_button, 'data-price')
            else:
                daten['price'] = ""
        else:
            daten['word_count'] = ""
            daten['price'] = ""
    else:
        for key in ['title', 'snippet', 'context_snippet', 'article_url', 'source_name_full', 'source_icon_url', 'word_count', 'price']:
            daten[key] = ""

    artikel_quelle_div = artikel_html_element.find('div', class_='article__source')
    if artikel_quelle_div:
        datum_div = artikel_quelle_div.find('div', class_='article__source__date')
        daten['date_published'] = hole_text_sicher(datum_div)
    else:
        daten['date_published'] = ""

    bereinigte_daten = {}
    for spalte in CSV_SPALTENNAMEN:
        bereinigte_daten[spalte] = daten.get(spalte, "")
    return bereinigte_daten

def scrape_tag_daten(ziel_datum_str, csv_schreiber, http_sitzung, api_basis_url, tages_parameter, min_v, max_v):
    """Verarbeitet das Scraping für einen einzelnen Tag, inklusive Paginierung."""
    print(f"\n--- Scrape für Datum: {ziel_datum_str} ---")
    aktueller_offset = 0
    artikel_heute_gefunden = 0

    while True:
        parameter = tages_parameter.copy()
        parameter['date'] = [f"from_{ziel_datum_str}", f"to_{ziel_datum_str}"]
        parameter['offset'] = aktueller_offset

        try:
            print(f"Rufe ab: {api_basis_url} mit Offset {aktueller_offset} für Datum {ziel_datum_str}")
            # Header werden von der Session genommen
            antwort = http_sitzung.get(api_basis_url, params=parameter, timeout=30) 
            antwort.raise_for_status()
            time.sleep(random.uniform(min_v, max_v))

            soup = BeautifulSoup(antwort.content, 'html.parser')
            artikel_auf_seite = soup.find_all('article', class_='article element')

            if not artikel_auf_seite:
                if aktueller_offset == 0:
                    print(f"Keine Artikel für {ziel_datum_str} gefunden.")
                else:
                    print(f"Keine weiteren Artikel auf Folgeseiten für {ziel_datum_str}. Wechsle zum nächsten Datum.")
                break

            print(f"{len(artikel_auf_seite)} Artikel auf dieser Seite gefunden (Offset {aktueller_offset}). Parse Daten...")
            artikel_heute_gefunden += len(artikel_auf_seite)

            for artikel_html in artikel_auf_seite:
                extrahierte_daten = extrahiere_artikel_daten(artikel_html, api_basis_url, ziel_datum_str)
                csv_schreiber.writerow(extrahierte_daten)
            
            aktueller_offset += parameter['size']
            if aktueller_offset >= 10000:
                print(f"10.000 Ergebnisse-Limit für Datum {ziel_datum_str} erreicht. Wechsle zum nächsten Datum.")
                break

        except requests.exceptions.HTTPError as e:
            print(f"HTTP-Fehler für {ziel_datum_str} bei Offset {aktueller_offset}: {e}")
            print("Möglicherweise Ende der Ergebnisse mit Fehlerseite erreicht oder blockiert. Wechsle zum nächsten Datum.")
            break 
        except requests.exceptions.RequestException as e:
            print(f"Anfrage fehlgeschlagen für {ziel_datum_str} bei Offset {aktueller_offset}: {e}")
            print("Versuch wird übersprungen. Für robustere Lösung Retries implementieren.")
            break 
        except Exception as e:
            print(f"Ein unerwarteter Fehler ist beim Verarbeiten von {ziel_datum_str} bei Offset {aktueller_offset} aufgetreten: {e}")
            import traceback
            traceback.print_exc()
            break 
    
    print(f"Scraping für {ziel_datum_str} abgeschlossen. Artikel heute gefunden: {artikel_heute_gefunden}")
    return artikel_heute_gefunden

## Hauptausführung des Skripts

In [ ]:
print(f"Starte Scraping für '{SUCHBEGRIFF}' von {START_DATUM_STR} bis {ENDE_DATUM_STR}")
print(f"Ausgabe wird gespeichert in: {AUSGABE_CSV_DATEI}")

gesamter_datumsbereich = generiere_datumsbereich(START_DATUM_STR, ENDE_DATUM_STR)
zu_verarbeitende_daten = []

letztes_datum_aus_csv_str = get_zuletzt_gescrapetes_datum(AUSGABE_CSV_DATEI)

if letztes_datum_aus_csv_str:
    print(f"Fortsetzung erkannt. Letztes verarbeitetes Datum in CSV: {letztes_datum_aus_csv_str}")
    letztes_datum_dt = datetime.strptime(letztes_datum_aus_csv_str, "%d.%m.%Y").date()
    for datum_str_check in gesamter_datumsbereich:
        aktuelles_pruefdatum_dt = datetime.strptime(datum_str_check, "%d.%m.%Y").date()
        if aktuelles_pruefdatum_dt > letztes_datum_dt:
            zu_verarbeitende_daten.append(datum_str_check)
    if not zu_verarbeitende_daten and letztes_datum_dt >= datetime.strptime(ENDE_DATUM_STR, "%d.%m.%Y").date():
         print("Alle Daten im angegebenen Bereich scheinen bereits verarbeitet worden zu sein.")
    elif not zu_verarbeitende_daten and letztes_datum_dt < datetime.strptime(ENDE_DATUM_STR, "%d.%m.%Y").date():
        print(f"Keine neueren Daten nach {letztes_datum_aus_csv_str} im definierten Bereich gefunden. Überprüfen Sie den Datumsbereich oder ob der letzte Tag unvollständig war (wird nicht automatisch nachgeholt).")
else:
    print("Keine existierende CSV-Datei gefunden oder keine Daten darin. Starte von vorne.")
    zu_verarbeitende_daten = gesamter_datumsbereich

if not zu_verarbeitende_daten and not (letztes_datum_aus_csv_str and datetime.strptime(letztes_datum_aus_csv_str, "%d.%m.%Y").date() >= datetime.strptime(ENDE_DATUM_STR, "%d.%m.%Y").date()) :
    if letztes_datum_aus_csv_str is None: 
        print("Keine Daten zum Verarbeiten basierend auf dem initialen Datumsbereich.")

gesamtzahl_artikel_gesammelt = 0
if zu_verarbeitende_daten:
    print(f"{len(zu_verarbeitende_daten)} Tage werden verarbeitet.")
    datei_existiert_bereits = os.path.exists(AUSGABE_CSV_DATEI) and os.path.getsize(AUSGABE_CSV_DATEI) > 0

    with requests.Session() as sitzung:
        sitzung.headers.update(HTTP_HEADERS)
        with open(AUSGABE_CSV_DATEI, 'a', newline='', encoding='utf-8') as csvfile:
            schreiber = csv.DictWriter(csvfile, fieldnames=CSV_SPALTENNAMEN)
            if not datei_existiert_bereits:
                schreiber.writeheader()
                print("CSV-Header geschrieben.")
            
            for idx, datum_str in enumerate(zu_verarbeitende_daten):
                print(f"Fortschritt: Tag {idx+1} von {len(zu_verarbeitende_daten)}")
                artikel_an_diesem_tag = scrape_tag_daten(
                    datum_str, 
                    schreiber, 
                    sitzung, 
                    BASIS_URL, 
                    BASIS_PARAMETER, 
                    MIN_VERZOEGERUNG, 
                    MAX_VERZOEGERUNG
                )
                # Gesamtzahl wird nicht mehr hier summiert, da bei Fortsetzung ungenau.
                # Zählung am Ende aus der Datei ist genauer.
                csvfile.flush()
else:
    if not (letztes_datum_aus_csv_str and datetime.strptime(letztes_datum_aus_csv_str, "%d.%m.%Y").date() >= datetime.strptime(ENDE_DATUM_STR, "%d.%m.%Y").date()):
      print("Keine Tage zur Verarbeitung übrig oder initialer Datumsbereich leer.")

print(f"\n--- Scraping beendet ---")
if os.path.exists(AUSGABE_CSV_DATEI):
    final_row_count = 0
    try:
        with open(AUSGABE_CSV_DATEI, 'r', newline='', encoding='utf-8') as f_count:
            reader_count = csv.reader(f_count)
            header = next(reader_count, None) # Header überspringen
            if header:
                 final_row_count = sum(1 for _ in reader_count)
        print(f"Insgesamt {final_row_count} Artikel-Datensätze in {AUSGABE_CSV_DATEI} gespeichert.")
        print("Hinweis: Diese Zahl kann Duplikate enthalten, falls Tage aufgrund von Abbrüchen erneut gescraped wurden.")
        print("Es wird empfohlen, die CSV-Datei nachträglich zu deduplizieren (z.B. mit Pandas).")
    except Exception as e:
        print(f"Konnte die finale Zeilenzahl nicht ermitteln: {e}")
else:
    print("Keine CSV-Datei erstellt, möglicherweise wurden keine Daten gefunden oder es gab Fehler.")

# Beispiel für nachträgliche Deduplizierung mit Pandas (auskommentiert):
# import pandas as pd
# try:
#     df = pd.read_csv(AUSGABE_CSV_DATEI)
#     print(f"DataFrame vor Deduplizierung: {df.shape}")
#     # Annahme: 'document_id_attr' und 'query_date' sind gute Schlüssel für Eindeutigkeit
#     # Wenn ein Artikel (gleiche ID) an einem Tag mehrmals vorkommt (durch teilweisen Rescrape), wird er entfernt.
#     df_deduplicated = df.drop_duplicates(subset=['document_id_attr', 'query_date'], keep='first')
#     print(f"DataFrame nach Deduplizierung: {df_deduplicated.shape}")
#     # Optional: Bereinigte Daten speichern
#     # df_deduplicated.to_csv(f"deduplicated_{AUSGABE_CSV_DATEI}", index=False, encoding='utf-8')
#     # print(f"Deduplizierte Daten gespeichert in: deduplicated_{AUSGABE_CSV_DATEI}")
# except FileNotFoundError:
#     print(f"CSV-Datei {AUSGABE_CSV_DATEI} nicht gefunden für Pandas-Operation.")
# except Exception as e:
#     print(f"Fehler bei der Pandas-Verarbeitung: {e}")

## Erläuterung des Codes

1.  **Importe:** Notwendige Bibliotheken, inklusive `re` für reguläre Ausdrücke (Textbereinigung).
2.  **Konfiguration:** Globale Variablen.
3.  **Hilfsfunktionen:**
    *   `generiere_datumsbereich`, `hole_text_sicher`, `hole_attribut_sicher`: Unverändert.
    *   `get_zuletzt_gescrapetes_datum`: Unverändert; findet das letzte *Datum*, für das *alle* Paginierungsseiten erfolgreich abgerufen wurden (oder zumindest der Versuch dafür endete).
    *   `bereinige_snippet_text`: Neue Funktion zur Säuberung von Text-Snippets (entfernt z.B. führende/folgende '..').
4.  **Funktionen für das Scraping:**
    *   `extrahiere_artikel_daten`:
        *   **Context Snippet:** Extrahiert nun robuster. Es sucht zuerst nach `div.article__text__trefferumgebung`. Zusätzlich (oder alternativ) durchsucht es den `div.article__text__panelBody__details` nach allen `div`s, die `<mark>`-Tags enthalten (außer Titel und Haupt-Snippet selbst), um alle Kontexte zu sammeln. Die gesammelten Kontexte werden mit ` ..//.. ` verbunden.
        *   **Quellen-Infos:** Die Logik zur Extraktion von `source_name_full` und `source_icon_url` aus dem `article__text__panelHeader` wurde verfeinert, um die verschiedenen möglichen Verschachtelungen (mit/ohne Icon-Div) besser zu handhaben.
        *   Die Extraktion von `word_count` und `price` wurde in den `article__text__bar`-Block verschoben, da sie dort strukturell hingehören.
        *   Verwendet `bereinige_snippet_text` für `snippet` und `context_snippet`.
    *   `scrape_tag_daten`: Übergibt nun nicht mehr `standard_header` explizit, da die `http_sitzung` (Session) bereits die Header enthält.
5.  **Hauptausführung des Skripts:**
    *   Die Fortsetzungslogik ist im Kern gleich geblieben: Es werden Tage *nach* dem letzten erfolgreich in der CSV gefundenen `query_date` verarbeitet.
    *   **Wichtig bzgl. Dopplungen:** Wenn ein Tag nicht vollständig gescraped wurde (z.B. bei Abbruch), wird dieser Tag beim nächsten Lauf *komplett neu* gescraped. Dies führt zu Duplikaten für die bereits erfassten Artikel dieses Tages. 
    *   Am Ende wird die Gesamtzahl der Zeilen in der CSV gezählt und ein expliziter Hinweis auf mögliche Duplikate und die Notwendigkeit der nachträglichen Deduplizierung gegeben.
    *   Ein auskommentiertes Beispiel für die Deduplizierung mit Pandas wurde hinzugefügt.

6.  **Benutzung im Jupyter Notebook (.ipynb):** Unverändert.

7.  **Wichtige Überlegungen:**
    *   **Deduplizierung:** Es ist **essenziell**, die resultierende CSV-Datei nachträglich zu deduplizieren, um saubere Daten für die Analyse zu erhalten. Das Pandas-Beispiel am Ende des Skripts zeigt einen Weg dafür.
    *   **Robustheit der Kontext-Extraktion:** Die neue Methode ist deutlich robuster, aber bei sehr komplexen oder unerwarteten HTML-Änderungen könnten immer noch Anpassungen nötig sein.